In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_experimental-0.3.4-py3-none-any.whl (209 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.8 MB 2.1 MB/s eta 0:00:02
   ------------- -------------------------- 1.3/3.8 MB 2.3 MB/s eta 0:00:02
   ---------------- ----------------------- 1.6/3.8 MB 2.1 MB/s eta 0:00:02
   --------------------- ------------------ 2.1/3.8 MB 2.1 MB/s eta 0:00:01
   ------------------------ --------------- 2.4/3.8 MB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 2.9/3.8 MB 2.0 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.8 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 2.1 MB/s eta 0:00:00
   --------------------------

## Built-in Tool - DuckDuckGo Search

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

India responds: In the aftermath of a deadly terrorist attack in Pahalgam that killed 26 people, India has suspended the Indus Waters Treaty, besides downgrading diplomatic ties with Pakistan, citing evidence of cross-border involvement. Foreign Secretary Vikram Misri said the Cabinet Committee on Security (CCS), chaired by Prime Minister Narendra Modi, convened shortly after the attack to ... Police in Indian-administered Kashmir say they identified three suspects, two of whom are Pakistani, in Tuesday's Pahalgam attack that killed 26 people.; India has summoned Pakistan's top ... India News, Latest News India: Read Live Breaking News from India along with Latest News and Today's Top Headlines Online in Bharat. Stay Up-to-date with Top news in India, current headlines, photos & videos online, live coverage on entertainment, business, politics, sports, technology and more at indianexpress.com. Headlines today: Get news headlines from India and around the world. Today's breaking news on

In [3]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [10]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('cd')

print(results)

Executing command:
 cd
c:\Users\sunil\Desktop\Langchain_tools



C:\Users\sunil\AppData\Roaming\Python\Python313\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

In [12]:
from langchain_core.tools import tool

In [13]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [14]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [15]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [17]:
result = multiply.invoke({"a":3, "b":9})

In [18]:
print(result)

27


In [19]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [20]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [21]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [22]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [23]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [24]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [26]:
result = multiply_tool.invoke({'a':30, 'b':13})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

390
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [27]:
from langchain.tools import BaseTool
from typing import Type

In [28]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [29]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [30]:
multiply_tool = MultiplyTool()

In [31]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [32]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [33]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [34]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
